### EIA API


In [1]:
# Energy Production and Usage
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### Crude Oil Production

https://www.eia.gov/dnav/pet/pet_crd_crpdn_adc_mbblpd_m.htm

In [2]:
# Crude Oil by State Group/Area
s_ids = ['MCRFPCO2', 'MCRFPTX2', 'MCRFPND2', 'MCRFPNM2', 
         'MCRFPUS2', 'MCRFP3FM2']
sl = []
for s in s_ids:
    sl.append(f'&facets[series][]={s}')
    
idlist = ''.join(sl)

url = ('https://api.eia.gov/v2/petroleum/crd/crpdn/data/'+
       f'?api_key={eia_key}'+       
       f'&frequency=monthly&data[0]=value{idlist}'+
       '&start=1989-01'+
       '&sort[0][column]=period&sort[0][direction]=desc'+
       '&offset=0&length=5000')

url2 = ('https://api.eia.gov/v2/petroleum/sum/snd/data/'+
       f'?api_key={eia_key}'+
       f'&frequency=monthly&data[0]=value{idlist}'+
       '&start=1989-01'+
       '&sort[0][column]=period&sort[0][direction]=desc'+
       '&offset=0&length=5000')

r = requests.get(url).json()
r2 = requests.get(url2).json()

d = {}
for s in s_ids:
    m = {pd.to_datetime(f'{i["period"]}-01'): i['value'] 
      for i in r['response']['data'] if i['series'] == s}
    d[s] = m
    m2 = {pd.to_datetime(f'{i["period"]}-01'): i['value'] 
      for i in r2['response']['data'] if i['series'] == s}
    if len(m) == 0:
        d[s] = m2
        
df = pd.DataFrame(d).sort_index().astype(float)

df['ND_NM_CO'] = df[['MCRFPND2', 'MCRFPNM2', 'MCRFPCO2']].sum(axis=1)
df['USA'] = df['MCRFPUS2']
df['TX'] = df['MCRFPTX2']
df['GM'] = df['MCRFP3FM2']
df['OTH'] = df['USA'] - df['TX'] - df['ND_NM_CO'] - df['GM']
data = df.loc['1989':,['ND_NM_CO', 'TX', 'OTH', 'GM', 'USA']].divide(1000, axis=1)
data.to_csv(data_dir / 'oil_prod.csv', index_label='date')

In [3]:
data.to_csv(data_dir / 'oil_prod.csv', index_label='date')
ltdt = dtxt(data.index[-1])['mon1']
prdt = dtxt(data.index[-13])['mon1']
val = pd.Series({'ltval': data['USA'].iloc[-1], 
           'pryrval': data['USA'].iloc[-13]})
fval = val.apply('{:.1f} million barrels per day'.format)
text = (f'During {ltdt}, the US produced {fval.ltval}, '+
        f'compared to {fval.pryrval} in {prdt}.')
write_txt(text_dir / 'oil_prod.txt', text)
print(text)

During March 2024, the US produced 13.2 million barrels per day, compared to 12.8 million barrels per day in March 2023.


### Electricity Production by Source

In [4]:
# Electricity Production by Major Source
s_list = ['COW', 'NUC', 'TSN', 'HYC', 'HPS', 'NG', 'WND', 'GEO', 'WWW',
         'WAS', 'PEL', 'PC', 'ALL', 'AOR', 'OTH', 'SUN', 'OOG']

df = pd.DataFrame()
for s in s_list:
    url = ('https://api.eia.gov/v2/electricity/electric-power-operational-data/data/'+
           f'?api_key={eia_key}&frequency=monthly&data[0]=generation'+
           f'&facets[fueltypeid][]={s}&facets[location][]=US&facets[sectorid][]=99'+
           '&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000')
    r = requests.get(url).json()
    df[s] = (pd.Series({pd.to_datetime(f'{i["period"]}-01'): i['generation'] 
                        for i in r['response']['data']})
          .sort_index().to_frame(name='VALUE')).astype('float')
    
df.to_csv(data_dir / 'elec_prod_raw.csv', index_label='date')

In [5]:
df = pd.read_csv(data_dir / 'elec_prod_raw.csv', index_col='date', 
                 parse_dates=True)

renew = ['SUN', 'HYC', 'WND', 'GEO', 'WAS', 'WWW']
renewables = [i for i in renew]
df['Renewables'] = df[renewables].sum(axis=1)

poth = ['PEL', 'PC', 'OTH', 'OOG', 'HPS']
petoth = [i for i in poth]
df['Petroleum and other'] = df[petoth].sum(axis=1)

bio = ['WWW', 'WAS']
biomass = [i for i in bio]
df['Biomass'] = df[biomass].sum(axis=1)

rename = {'COW': 'Coal', 
          'NUC': 'Nuclear', 
          'NG': 'Natural Gas',
          'HYC': 'Hydroelectric',
          'SUN': 'Solar',
          'WND': 'Wind',
          'GEO': 'Geothermal'}
df.rename(rename, axis=1, inplace=True)

var = ['Coal', 'Natural Gas', 'Nuclear', 'Renewables', 
       'Petroleum and other', 'Hydroelectric', 'Solar', 
       'Wind', 'Geothermal', 'Biomass']
data = df[var].rolling(12).sum().dropna().divide(1000, axis=1)
data.to_csv(data_dir / 'elec_prod.csv', index_label='date')

popgdp = pd.read_csv(data_dir / 'gdpjobslvl.csv', index_col='date', 
                     parse_dates=True)
popgdp.index = popgdp.index + pd.DateOffset(months=2)
pop = popgdp['POP'].dropna()
popch = (pop.iloc[-1] / pop.loc['2011-03-01'].mean() - 1) * 100
popinc = inc_dec_percent(popch)
gdp = popgdp['GDP'].dropna()
gdpch = (gdp.iloc[-1] / gdp.loc['2011-03-01'].mean() - 1) * 100
gdpinc = inc_dec_percent(gdpch)
major = ['Coal', 'Natural Gas', 'Nuclear', 'Renewables', 
         'Petroleum and other']
elec = data[major].sum(axis=1)
df2 = pd.DataFrame()
df2['elec'] = elec
df2['gdp'] = gdp
df2['pop'] = pop
df2['elecgdp'] = elec / gdp
df2['elecpop'] = elec / pop
df2 = ((df2 / df2.loc['2011-03-01']) - 1) * 100
df2.loc['2011-03-01':].to_csv(data_dir / 'elec.csv', index_label='date')

colors = {'gdp': 'green!60!blue', 
          'pop': 'blue!90!black', 
          'elec': 'red', 
          'elecpop': 'orange', 
          'elecgdp': 'cyan'}

adj = node_adj(df2)
node_file = open(text_dir / 'elec_nodes.txt', 'w')
for series, color in colors.items():
    if series in adj.keys():
        offset = adj[series]
    else:
        offset = 0
    node_file.write(end_node(df2[series].dropna(), color, 
                             percent=True))
node_file.close()
elecgdpch = value_text(df2.elecgdp.dropna().iloc[-1], 'increase_by')
elch = df2.elec.dropna().iloc[-1]
ctxt = ('remained fairly constant at around four trillion '+
        'kilowatt hours')
electxt = value_text(elch) if abs(elch) > 0.5 else ctxt
ltdate = dtxt(df.index[-1])['mon1']
ltval = data[major].iloc[-1].sum()
text = ('Since 2011, annualized total US \\textbf{electricity '+
        f'generation}} has {electxt}. '+
        f'Over the same period, the US population has {popinc} '+
        f'{c_line(colors["pop"])} and real GDP has {gdpinc} '+
        f'{c_line(colors["gdp"])}. As a result, the electricity '+
        f'required to produce a unit of real GDP {elecgdpch} '+
        f'{c_line(colors["elecgdp"])}.') 
write_txt(text_dir / 'elec_prod.txt', text)
print(text)

ngval = data['Natural Gas'].iloc[-1]
coalval = data['Coal'].iloc[-1]
nucval = data['Nuclear'].iloc[-1]
renval = data['Renewables'].iloc[-1]
text = (f'During the 12 months ending {ltdate}, the '+
        f'US generated {ltval:,.0f} billion kilowatt '+
        f'hours of electricity. Of this, {ngval:,.0f} billion '+
        'kilowatt hours were generated using '+
        'natural gas (see\cbox{blue!40!cyan!60!white}), '+
        f'{coalval:,.0f} billion kilowatt hours were generated '+
        f'from coal (see\cbox{{brown}}), {nucval:,.0f} billion '+
        'from nuclear (see\cbox{{red!80!blue!70!white}}), and '+
        f'{renval:,.0f} billion from renewable sources '+
        '(see\cbox{{green!75!blue}}).')
write_txt(text_dir / 'elec_prod2.txt', text)
print(text)

hydval = data['Hydroelectric'].iloc[-1]
bioval = data['Biomass'].iloc[-1]
geoval = data['Geothermal'].iloc[-1]
sunval = data['Solar'].iloc[-1]
wndval = data['Wind'].iloc[-1]
text = (f'Among renewable energy sources, over the year ending '+
        f'{ltdate}, {hydval:,.0f} billion kilowatt hours of '+
        'electricity were generated with conventional hydroelectric '+
        f'(see\cbox{{cyan}}), {bioval:,.0f} billion kilowatt hours '+
        f'were generated from biomass (see\cbox{{brown!92!black}}), '+
        f'{geoval:,.0f} billion were generated from geothermal '+
        f'(see\cbox{{orange}}), {wndval:,.0f} billion from '+
        f'wind (see\cbox{{green!65!blue}}), and {sunval:,.0f} '+
        f'billion from solar (see\cbox{{yellow!70!orange}}).')
write_txt(text_dir / 'elec_prod3.txt', text)
print(text)

Since 2011, annualized total US \textbf{electricity generation} has increased 2.1 percent. Over the same period, the US population has increased by 8.4 percent (see {\color{blue!90!black}\textbf{---}}) and real GDP has increased by 34.5 percent (see {\color{green!60!blue}\textbf{---}}). As a result, the electricity required to produce a unit of real GDP decreased by 24.0 percent (see {\color{cyan}\textbf{---}}).
During the 12 months ending March 2024, the US generated 4,215 billion kilowatt hours of electricity. Of this, 1,830 billion kilowatt hours were generated using natural gas (see\cbox{blue!40!cyan!60!white}), 676 billion kilowatt hours were generated from coal (see\cbox{brown}), 778 billion from nuclear (see\cbox{{red!80!blue!70!white}}), and 900 billion from renewable sources (see\cbox{{green!75!blue}}).
Among renewable energy sources, over the year ending March 2024, 242 billion kilowatt hours of electricity were generated with conventional hydroelectric (see\cbox{cyan}), 47 b

### Electricity Retail Sales by Sector

In [6]:
# Electricity Retail Sales by Sector
s_list = ['RES', 'COM', 'IND']
df = pd.DataFrame()
for s in s_list:
    url = (f'https://api.eia.gov/v2/electricity/retail-sales/data/?api_key={eia_key}&'+
           f'frequency=monthly&data[0]=sales&facets[sectorid][]={s}'+
           '&facets[stateid][]=US&sort[0][column]=period&sort[0][direction]=desc'+
           '&offset=0&length=5000')
    r = requests.get(url).json()
    df[s] = (pd.Series({pd.to_datetime(f'{i["period"]}-01'): i['sales'] 
                        for i in r['response']['data']})
          .sort_index().to_frame(name='VALUE')).astype('float')
df.to_csv(data_dir / 'elec_sales_raw.csv', index_label='date')

In [7]:
df = pd.read_csv(data_dir / 'elec_sales_raw.csv', index_col='date', 
                 parse_dates=True) / 1_000

rename = {'RES': 'Residential', 
          'COM': 'Commercial',
          'IND': 'Industrial'}
df.rename(rename, axis=1, inplace=True)

data = df.rolling(12).sum().dropna()
data.to_csv(data_dir / 'elec_sales.csv', index_label='date')

cats = {'Commercial': 'blue!80!cyan', 'Industrial': 
        'black!35!white', 'Residential': 'green!90!black'}
nodes = '\n'.join([end_node(data[cat], col, digits='comma') 
                   for cat, col in cats.items()])
write_txt(text_dir / 'elec_ret_nodes.txt', nodes)

cl = {cat[:3]: c_line(col) for cat, col in cats.items()}
ltdt = dtxt(data.index[-1])['mon1']
dft = data.applymap('{:,.0f} billion'.format)
lt = dft.iloc[-1]
pr = dft.loc['2019-12-01']
text = (f'Over the year ending {ltdt}, retail sales of electricity '+
        f'to the residential sector total {lt.Residential} '+
        f'kilowatt hours, compared to {pr.Residential} '+
        f'during 2019 {cl["Res"]}. Commercial sector '+
        f'electricity sales total {lt.Commercial} kilowatt '+
        f'hours over the year ending {ltdt}, and {pr.Commercial} '+
        f'in 2019 {cl["Com"]}. Industrial '+
        f'sector sales total {lt.Industrial} kilowatt '+
        f'hours in the latest 12 months of data and {pr.Industrial} '+
        f'in 2019 {cl["Ind"]}.')
write_txt(text_dir / 'elec_ret.txt', text)
print(text)

Over the year ending March 2024, retail sales of electricity to the residential sector total 1,464 billion kilowatt hours, compared to 1,440 billion during 2019 (see {\color{green!90!black}\textbf{---}}). Commercial sector electricity sales total 1,382 billion kilowatt hours over the year ending March 2024, and 1,361 billion in 2019 (see {\color{blue!80!cyan}\textbf{---}}). Industrial sector sales total 1,029 billion kilowatt hours in the latest 12 months of data and 1,002 billion in 2019 (see {\color{black!35!white}\textbf{---}}).


### Weekly retail gasoline prices

In [8]:
s_id = 'EMM_EPM0_PTE_NUS_DPG'
url = (f'https://api.eia.gov/v2/petroleum/pri/gnd/data/?api_key={eia_key}'+
       f'&frequency=weekly&data[0]=value&facets[series][]={s_id}&start=1990-08-20'+
       '&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000')
r = requests.get(url).json()
df = (pd.Series({pd.to_datetime(i['period']): float(i['value']) 
           for i in r['response']['data']}).to_frame(name='Value').sort_index())
df.to_csv(data_dir / 'gas_price.csv', index_label='date')

node = end_node(df.Value, 'blue', date='d', digits=2, dollar=True, 
                full_year=True)
write_txt(text_dir / 'gas_price_node.txt', node)

# Text
ltdt = dtxt(df.index[-1])['day1']
ltval = df.Value.iloc[-1]
pryrval = df.Value.iloc[-53]
v19 = df.Value.loc['2019'].mean()
v1113 = df.Value.loc['2011': '2013'].mean()
cl = c_line('blue')
ch = value_text(df.Value.diff().iloc[-1], 'increase_of', ptype=None, 
                dollar=True, digits=2, threshold=0.01)
url = f'https://www.eia.gov/opendata/qb.php?category=241021&sdid={s_id}'
text = (f'On {ltdt}, the US average \href{{{url}}}{{price}} for a gallon of '+
        f'\\textbf{{gasoline}} is \${ltval:.2f} {cl}, {ch} from the week prior. '+
        'This gas price measure, which is the average across '+
        f'formulations, grades, and locations, was \${pryrval:.2f} one '+
        f'year prior, and averaged \${v19:.2f} in 2019. During 2011--2013, '+
        f'the average gas price was \${v1113:.2f}.')
write_txt(text_dir / 'gas_price.txt', text)
print(text)

On June 10, 2024, the US average \href{https://www.eia.gov/opendata/qb.php?category=241021&sdid=EMM_EPM0_PTE_NUS_DPG}{price} for a gallon of \textbf{gasoline} is \$3.55 (see {\color{blue}\textbf{---}}), a decrease of \$0.09 from the week prior. This gas price measure, which is the average across formulations, grades, and locations, was \$3.71 one year prior, and averaged \$2.69 in 2019. During 2011--2013, the average gas price was \$3.61.


### Oil prices (WTI)

In [9]:
data = pd.DataFrame()
for year in [1989, 2003, 2015]:
    url = (f'https://api.eia.gov/v2/petroleum/pri/fut/data/?api_key={eia_key}'+
           '&frequency=daily&data[0]=value&facets[series][]=RCLC1'+
           f'&start={year}-01-01&sort[0][column]=period'+
           '&sort[0][direction]=asc&offset=0&length=8000')

    r = requests.get(url).json()
    s = (pd.Series({pd.to_datetime(i['period']): float(i['value']) 
                 for i in r['response']['data']})
            .to_frame(name='VALUE').sort_index())
    data = pd.concat([data, s])

data = data[~data.index.duplicated(keep='first')]
print('Latest Data:', dtxt(data.index[-1])['day1'])
data.to_csv(data_dir / 'wti_raw.csv', index_label='date')

Latest Data: April 5, 2024


In [10]:
data = pd.read_csv(data_dir / 'wti_raw.csv', index_col='date', 
                   parse_dates=True)
df = data.resample('MS').mean().iloc[:-1]
df.index = df.index + pd.DateOffset(days=14)
lt = data.iloc[-1].to_frame().T
df = pd.concat([df, lt])
df.to_csv(data_dir / 'wti.csv', index_label='date')

oneyr = value_text(df.VALUE.pct_change(12).iloc[-1] * 100)
twoyr = value_text(df.VALUE.pct_change(48).iloc[-1] * 100)

color = 'red!80!purple'
node = end_node(df.VALUE, color, dollar=True, digits=2, date='d', 
                offset=True, full_year=True)
write_txt(text_dir / 'oil_node.txt', node)

maxdt = df.VALUE.idxmax()
maxdtt = dtxt(maxdt)['mon1']
maxval = df.VALUE.max()
ltch = df.loc[maxdt, 'VALUE'] - df.VALUE.iloc[-1]
ldate = dtxt(df.index[-1])['day1']
url = 'https://www.eia.gov/dnav/pet/hist/RCLC1D.htm'
text = (f'On {ldate}, the \href{{{url}}}{{futures price}} '+
        f'for a barrel of west Texas intermediate (WTI) '+
        f'\\textbf{{crude oil}} is \${df.VALUE.iloc[-1]:.2f} {c_line(color)}. '+
        f'Over the past year, this measure of oil prices {oneyr}. Over '+
        f'the past four years, the price {twoyr}. The WTI price is '+
        f'currently \${ltch:.0f} below its peak monthly '+
        f'average price of \${maxval:.0f} per barrel in {maxdtt}.')
write_txt(text_dir / 'wti.txt', text)
print(text)

On April 5, 2024, the \href{https://www.eia.gov/dnav/pet/hist/RCLC1D.htm}{futures price} for a barrel of west Texas intermediate (WTI) \textbf{crude oil} is \$86.91 (see {\color{red!80!purple}\textbf{---}}). Over the past year, this measure of oil prices increased 9.4 percent. Over the past four years, the price increased 412.0 percent. The WTI price is currently \$47 below its peak monthly average price of \$134 per barrel in June 2008.
